# Parametric fitting app for 1D DC resistivity (i.e. VES) data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from SimPEG.electromagnetics import resistivity as DC
from discretize import TensorMesh
from SimPEG import maps
from SimPEG import Data
from SimPEG.electromagnetics.static.resistivity import DCSimulation_1D, IO
from SimPEG.electromagnetics.static.utils import StaticUtils
from SimPEG import (
    maps, data_misfit, regularization,
    optimization, inverse_problem, inversion, directives, Data
)
from ipywidgets import widgets, interact, interactive
import warnings
warnings.filterwarnings("ignore")

In [2]:
def read_ves(fname):
    import pandas as pd
    df = pd.read_csv(fname)
    ab_2 = df['AB/2 (m)']
    mn_2 = df['MN/2 (m)']
    n_sounding = ab_2.size
    # We generate tx and rx lists:
    srclist = []
    for ii in range(n_sounding):
        a_loc = ab_2[ii]
        b_loc = -ab_2[ii]
        m_loc = mn_2[ii]
        n_loc = -mn_2[ii]
        rx = DC.Rx.Dipole(np.r_[m_loc, 0, 0], np.r_[n_loc, 0, 0])
        locA = np.r_[a_loc, 0, 0]
        locB = np.r_[b_loc, 0, 0]
        src = DC.Src.Dipole([rx], locA, locB)
        srclist.append(src)
    survey = DC.Survey(srclist)  
    data = Data(survey=survey, dobs=df['App. Res. (Ohm m)'].values)
    return data

def generate_resistivity_model(rho_1, rho_2, rho_3, t_1, t_2):
    hz = np.r_[t_1, t_2, 100.]
    rho = np.r_[rho_1, rho_2, rho_3]
    return hz, rho

def plot_resistivity(hz, rho):
    mesh_1d = TensorMesh([hz])
    StaticUtils.plot_layer(rho, mesh_1d)    

def simulate(hz, rho):
    wires = maps.Wires(('rho', rho.size), ('t', rho.size-1))
    mapping_rho = maps.ExpMap(nP=rho.size) * wires.rho
    mapping_t = maps.ExpMap(nP=rho.size-1) * wires.t
    # mapping_t = wires.t
    simulation = DCSimulation_1D(
        rhoMap=mapping_rho,
        thicknessesMap=mapping_t,    
        survey=data.survey, 
        data_type='apparent_resistivity'
    )    
    m = np.log(np.r_[rho, hz[:-1]])
    data_tmp = simulation.makeSyntheticData(m)
    ab = simulation.electrode_separations['AB']    
    return ab, data_tmp

def foo(rho_1, rho_2, rho_3, t_1, t_2):
    import matplotlib
    matplotlib.rcParams['font.size'] = 14    
    hz, rho = generate_resistivity_model(rho_1, rho_2, rho_3, t_1, t_2)
    mesh_1d = TensorMesh([hz])
    ab, data_tmp = simulate(hz, rho)
    fig, axs = plt.subplots(1,2,figsize=(10, 5))
    StaticUtils.plot_layer(rho, mesh_1d, ax=axs[0])    
    axs[1].loglog(ab/2., data_tmp.dobs, '-')   
    axs[1].loglog(ab/2., data.dobs, 'kx')      
    axs[1].set_xlabel("AB/2")
    axs[1].set_ylabel("App. resistivity (ohm-m)")   
    axs[1].grid(which='both')
    plt.tight_layout()

## Step1: Read VES data

In [3]:
fname = './assets/ves-obs-3.csv'
data = read_ves(fname)

## Step 2: Fit observed VES data using 3-layer app

In [4]:
q = interactive(
    foo,
    rho_1=widgets.IntSlider(value=np.median(data.dobs), min=1, max=5000, step=1, continuous_update=False, description='$\\rho_1$'),
    rho_2=widgets.IntSlider(value=np.median(data.dobs), min=1, max=5000, step=1, continuous_update=False, description='$\\rho_2$'),
    rho_3=widgets.IntSlider(value=np.median(data.dobs), min=1, max=5000, step=1, continuous_update=False, description='$\\rho_3$'),
    t_1=widgets.IntSlider(value=10, min=1, max=5000, step=1, continuous_update=False, description='$h_1$'),
    t_2=widgets.IntSlider(value=10, min=1, max=5000, step=1, continuous_update=False, description='$h_2$'),
)
q

interactive(children=(IntSlider(value=94, continuous_update=False, description='$\\rho_1$', max=5000, min=1), …

## Step 3: Set parameters of parametric inversion

In [92]:
rho0 = np.median(data.dobs)
thickness0 = 30.
n_layer = 8
maxIter = 10
floor = 10.
std = 0.01
alpha_s = 1
alpha_x = 1
beta0_ratio = 1.

In [93]:
m0 = np.r_[np.log(np.ones(n_layer)*rho0), np.log(np.ones(n_layer-1) * thickness0)]

wires = maps.Wires(('rho', n_layer), ('t', n_layer-1))
mapping_rho = maps.ExpMap(nP=n_layer) * wires.rho
mapping_t = maps.ExpMap(nP=n_layer-1) * wires.t
simulation = DCSimulation_1D(
    rhoMap=mapping_rho,
    thicknessesMap=mapping_t,    
    survey=data.survey, 
    data_type='apparent_resistivity'
)        

dmis = data_misfit.L2DataMisfit(simulation=simulation, data=data)
uncert = abs(data.dobs) * std + floor
dmis.W = 1./uncert

simulation = DCSimulation_1D(
    rhoMap=mapping_rho,
    thicknessesMap=mapping_t,    
    survey=data.survey, 
    data_type='apparent_resistivity'
)

mesh_rho = TensorMesh([n_layer])
reg_rho = regularization.Simple(
    mesh_rho, alpha_s=alpha_s, alpha_x=alpha_x,
    mapping=wires.rho
)
mesh_t = TensorMesh([n_layer-1])
reg_t = regularization.Simple(
    mesh_t, alpha_s=alpha_s, alpha_x=alpha_x,
    mapping=wires.t    
)
reg = reg_rho + reg_t
opt = optimization.InexactGaussNewton(
    maxIter=maxIter, maxIterCG=20, print_type='ubc'
)
invProb = inverse_problem.BaseInvProblem(dmis, reg, opt)
target = directives.TargetMisfit()
# Create an inversion object
beta = directives.BetaSchedule(coolingFactor=2., coolingRate=1.)
betaest = directives.BetaEstimate_ByEig(beta0_ratio=beta0_ratio)
save =  directives.SaveOutputDictEveryIteration()
inv = inversion.BaseInversion(invProb, directiveList=[beta, target, save, betaest])
# inv = inversion.BaseInversion(invProb, directiveList=[beta, target, save])
opt.remember('xc')
# rho_s = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
mopt = inv.run(m0)
n_iteration = len(save.outDict)

SimPEG.InvProblem will set Regularization.mref to m0.
SimPEG.InvProblem will set Regularization.mref to m0.

    SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
    ***Done using same Solver and solverOpts as the problem***
SimPEG.SaveOutputDictEveryIteration will save your inversion progress as dictionary: '###-InversionModel-2020-01-21-11-03.npz'
model has any nan: 0
=================================== Inexact Gauss Newton ===================================
  #     beta     phi_d     phi_m     phi_s     phi_x     phi_y     phi_z    LS    Comment   
--------------------------------------------------------------------------------------------
   0  2.56e+02  2.43e+03  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00   0              
   1  1.28e+02  1.50e+03  8.48e-01  7.33e-01  1.15e-01  0.00e+00  0.00e+00   0              
   2  6.41e+01  9.10e+02  2.23e+00  1.96e+00  2.75e-01  0.00e+00  0.00e+00   0              
   3  3.20e+01  1.68e+02  5.59e+00  4.99e+00  6.

In [100]:
def plot_invesion_results(iteration):
    import matplotlib
    matplotlib.rcParams['font.size'] = 14    
    rho_tmp = np.r_[q.kwargs['rho_1'], q.kwargs['rho_2'], q.kwargs['rho_3']]
    hz_tmp = np.r_[q.kwargs['t_1'], q.kwargs['t_2'], 100]
    mesh_tmp = TensorMesh([hz_tmp])
    mopt = save.outDict[iteration]['m']
    rho_est = mapping_rho * mopt
    t_est = mapping_t * mopt
    mesh_est = TensorMesh([np.r_[t_est, 100]])
    
    ab, data_tmp = simulate(hz_tmp, rho_tmp)
    fig, axs = plt.subplots(1,2,figsize=(10, 5))
    StaticUtils.plot_layer(rho_tmp, mesh_tmp, ax=axs[0], **{'color':'k', 'alpha':0.5, 'linestyle':'--'})    
    StaticUtils.plot_layer(rho_est, mesh_est, showlayers=False, **{'color':'r'}, ax=axs[0])    
    axs[1].loglog(ab/2., save.outDict[iteration]['dpred'], '-')   
    axs[1].loglog(ab/2., data.dobs, 'kx')      
    axs[1].loglog(ab/2., data_tmp.dobs, 'k--', alpha=0.5)          
    axs[1].set_xlabel("AB/2")
    axs[1].set_ylabel("App. resistivity (ohm-m)")   
    axs[1].grid(which='both')
    plt.tight_layout()

In [101]:
interact(plot_invesion_results, iteration=widgets.IntSlider(min=1, max=n_iteration, step=1))

interactive(children=(IntSlider(value=1, description='iteration', max=10, min=1), Output()), _dom_classes=('wi…

<function __main__.plot_invesion_results(iteration)>